## Set-Up

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
def mod_5(in_list):  #Input is a binary array [a, b, c ,d , e] encoding a number e + 2 d + 4 c + ...
                # Output is the encoded number mod 5 (as an int).
  return int("".join(str(x) for x in in_list[::-1]), 2) % 5

def div_5(in_list): 
  return int( bool( mod_5( in_list ) ) )


In [5]:
target_func = div_5
input_size = 20

In [6]:
def get_batch(batch_size):
  X = np.random.randint(0, 2, (batch_size, input_size), dtype = np.uint8) % 2
  y = np.apply_along_axis(mod_5, 1, X) 
  return (X[..., np.newaxis].astype(np.uint32), y)

#This time, output all the partial results:
def get_batch_out_seq(batch_size):

  X = np.random.randint(0, 2, (batch_size, input_size), dtype = np.uint8) % 2
  y = np.empty((batch_size, input_size), dtype = "uint64",)
  for i in range(input_size):
    y[:,i] = np.apply_along_axis(mod_5, 1, X[:, :i + 1])

  return (X[..., np.newaxis].astype(np.uint32), y)




## Network with GRU units

In [7]:
model = keras.models.Sequential([
  keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.Dense(5, activation = "softmax")
])

def last_time_step_scc(Y_true, Y_pred):
  m = tf.keras.metrics.SparseCategoricalAccuracy()
  m.update_state(Y_true[:, -1], Y_pred[:, -1])
  return m.result().numpy()


  return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss = "sparse_categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = [last_time_step_scc])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 20)          1380      
                                                                 
 gru_1 (GRU)                 (None, None, 20)          2520      
                                                                 
 dense (Dense)               (None, None, 5)           105       
                                                                 
Total params: 4,005
Trainable params: 4,005
Non-trainable params: 0
_________________________________________________________________


## Generate data and fit

In [8]:
X_train, y_train = get_batch_out_seq(70000)
X_valid, y_valid = get_batch_out_seq(10000)
X_test, y_test = get_batch_out_seq(2000)


In [9]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

model.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid),
          callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)])

Epoch 1/20
2188/2188 [==============================] - 48s 20ms/step - loss: 1.1261 - accuracy: 0.4835 - val_loss: 0.6390 - val_accuracy: 0.7310
Epoch 2/20
2188/2188 [==============================] - 45s 21ms/step - loss: 0.4567 - accuracy: 0.8139 - val_loss: 0.3455 - val_accuracy: 0.8610
Epoch 3/20
2188/2188 [==============================] - 45s 21ms/step - loss: 0.2908 - accuracy: 0.8841 - val_loss: 0.2478 - val_accuracy: 0.9032
Epoch 4/20
2188/2188 [==============================] - 45s 21ms/step - loss: 0.2202 - accuracy: 0.9145 - val_loss: 0.1974 - val_accuracy: 0.9241
Epoch 5/20
2188/2188 [==============================] - 44s 20ms/step - loss: 0.1776 - accuracy: 0.9314 - val_loss: 0.1632 - val_accuracy: 0.9374
Epoch 6/20
2188/2188 [==============================] - 45s 20ms/step - loss: 0.1455 - accuracy: 0.9453 - val_loss: 0.1306 - val_accuracy: 0.9504
Epoch 7/20
2188/2188 [==============================] - 44s 20ms/step - loss: 0.1211 - accuracy: 0.9549 - val_loss: 0.1128 -

Works quite nicely! Let's try different network architectures...

## Different architectures

In [10]:
# Only one layer
model_2 = keras.models.Sequential([
  keras.layers.GRU(20, return_sequences=True, input_shape = [None, 1]),
  keras.layers.Dense(5, activation = "softmax")
])

model_2.compile(loss = "sparse_categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

model_2.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid), # 0.999 accuracy after 20 epochs
            callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]) 

Epoch 1/20
2188/2188 [==============================] - 26s 11ms/step - loss: 1.4139 - accuracy: 0.3348 - val_loss: 1.1859 - val_accuracy: 0.4639
Epoch 2/20
2188/2188 [==============================] - 24s 11ms/step - loss: 1.0403 - accuracy: 0.5408 - val_loss: 0.9012 - val_accuracy: 0.6119
Epoch 3/20
2188/2188 [==============================] - 24s 11ms/step - loss: 0.7517 - accuracy: 0.6812 - val_loss: 0.5962 - val_accuracy: 0.7493
Epoch 4/20
2188/2188 [==============================] - 23s 11ms/step - loss: 0.4657 - accuracy: 0.8014 - val_loss: 0.3578 - val_accuracy: 0.8559
Epoch 5/20
2188/2188 [==============================] - 24s 11ms/step - loss: 0.2754 - accuracy: 0.9009 - val_loss: 0.1927 - val_accuracy: 0.9447
Epoch 6/20
2188/2188 [==============================] - 24s 11ms/step - loss: 0.1227 - accuracy: 0.9719 - val_loss: 0.0674 - val_accuracy: 0.9896
Epoch 7/20
2188/2188 [==============================] - 25s 12ms/step - loss: 0.0380 - accuracy: 0.9956 - val_loss: 0.0182 -

In [12]:
# Only one layer, less cells
model_3 = keras.models.Sequential([
  keras.layers.GRU(10, return_sequences=True, input_shape = [None, 1]),
  keras.layers.Dense(5, activation = "softmax")
])

model_3.compile(loss = "sparse_categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

model_3.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid), # 0.64 accuracy after 20 epochs
            callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]) 

Epoch 1/20
2188/2188 [==============================] - 26s 11ms/step - loss: 1.4868 - accuracy: 0.2938 - val_loss: 1.3420 - val_accuracy: 0.3768
Epoch 2/20
2188/2188 [==============================] - 23s 10ms/step - loss: 1.2070 - accuracy: 0.4434 - val_loss: 1.1261 - val_accuracy: 0.4764
Epoch 3/20
2188/2188 [==============================] - 23s 11ms/step - loss: 1.0928 - accuracy: 0.4952 - val_loss: 1.0594 - val_accuracy: 0.5152
Epoch 4/20
2188/2188 [==============================] - 24s 11ms/step - loss: 1.0490 - accuracy: 0.5175 - val_loss: 1.0275 - val_accuracy: 0.5292
Epoch 5/20
2188/2188 [==============================] - 25s 11ms/step - loss: 1.0240 - accuracy: 0.5276 - val_loss: 1.0077 - val_accuracy: 0.5385
Epoch 6/20
2188/2188 [==============================] - 23s 11ms/step - loss: 1.0054 - accuracy: 0.5378 - val_loss: 0.9911 - val_accuracy: 0.5456
Epoch 7/20
2188/2188 [==============================] - 23s 11ms/step - loss: 0.9876 - accuracy: 0.5478 - val_loss: 0.9728 -

In [ ]:
model_3.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid), 
            callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]) #No improvement anymore, add training data
X_train, y_train = get_batch_out_seq(70000)

model_3.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid), 
            callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]) 



In [17]:
X_train, y_train = get_batch_out_seq(70000)
model_3.compile(loss = "sparse_categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.003), metrics = ["accuracy"])

model_3.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid), 
            callbacks = [tf.keras.callbacks.EarlyStopping(patience=2)]) 

Epoch 1/20
2188/2188 [==============================] - 28s 12ms/step - loss: 0.7922 - accuracy: 0.6611 - val_loss: 0.7814 - val_accuracy: 0.6651
Epoch 2/20
2188/2188 [==============================] - 25s 12ms/step - loss: 0.7840 - accuracy: 0.6644 - val_loss: 0.7279 - val_accuracy: 0.6811
Epoch 3/20
2188/2188 [==============================] - 27s 12ms/step - loss: 0.8599 - accuracy: 0.6238 - val_loss: 1.0786 - val_accuracy: 0.5206
Epoch 4/20
2188/2188 [==============================] - 26s 12ms/step - loss: 0.9927 - accuracy: 0.5655 - val_loss: 0.8961 - val_accuracy: 0.6050
